# Valeur foncière moyenne : définition d'un loyer au m²


In [2]:
# import all the libraries needed
import statistics
import requests
import json
import math

In [3]:
rayon = 50
lat = 48.85  # latitude
lon = 2.35  # longitude


def valeur_fonciere_au_m2(rayon, lat, lon):

    params = {"format": "json"}
    response = requests.get(
        f"https://api.cquest.org/dvf?lat={lat}&lon={lon}&dist={rayon}", params=params)  # requête de l'API

    def jprint(obj):
        # création d'un string à partir de l'objet JSON
        text = json.dumps(obj, indent=4)
        print(text)

    # tableau où on stocke les valeurs foncières se trouvant dans le périmètre choisi
    valeur_fonciere_moyenne = []

    valeur_api = response.json()  # la valeur foncière que l'on cherche avec l'API

    for i in range(len(valeur_api["features"])):
        # Recherche de surface du logement
        surface = (valeur_api["features"][i]
                   ["properties"]["surface_relle_bati"])
        if surface != 0:
            # Recherche de la valeur foncière
            prix = (valeur_api["features"][i]["properties"]["valeur_fonciere"])
            prix_au_m2 = prix/surface
            valeur_fonciere_moyenne.append(prix_au_m2)

    return round(statistics.mean(valeur_fonciere_moyenne))

In [4]:
# print(f"La valeur foncière moyenne sur un rayon de {rayon}m est de {valeur_fonciere_au_m2(rayon, lat, lon)} euros par m2.\n")

# Coûts de construction


## Estimation peu fiable


In [5]:
def cout_construction_immeuble1(surface, lon, lat):
    assert (surface >= 0)

    surface += surface/10  # surface totale = surface logement + partie commune ≈ 10%

    # Exemple rayon de 20m pour le calcul de valeur_fonciere_au_m2
    prix_loyer = valeur_fonciere_au_m2(20, lon, lat)*surface

    prix_etudes_de_sol = prix_loyer*0.02
    prix_controles_techniques = prix_loyer*0.03
    prix_honoraires_architectes = prix_loyer*0.08
    return round(prix_loyer + prix_etudes_de_sol + prix_controles_techniques + prix_honoraires_architectes)

## Estimation plus fiable


##### 1) Calcul des mesures linéaires, surface, volume


In [49]:
def demander_dimensions_immeuble():
    longueur = float(input("Longueur (en m) : "))
    largeur = float(input("Largeur (en m) : "))
    hauteur = float(input("Hauteur (en m) : "))
    return [longueur, largeur, hauteur]


def demander_dimensions_maison():
    longueur_base = float(input("Longueur de la base (en m) : "))
    largeur_base = float(input("Largeur de la base (en m) : "))
    hauteur_base = float(input("Hauteur de la base (en m) : "))
    longueur_toit = float(input("Longueur du toit (en m) : "))
    largeur_toit = float(input("Largeur du toit (en m) : "))
    hauteur_toit = float(input("Hauteur du toit (en m) : "))
    return [longueur_base, largeur_base, hauteur_base, longueur_toit, largeur_toit, hauteur_toit]


ratio_efficacite_etage = 0.83


def calculer_mesures(type_batiment):
    # Calcul des mesures en fonction du type de bâtiment
    if type_batiment == "immeuble":
        dimensions = demander_dimensions_immeuble()

        longueur = dimensions[0]
        largeur = dimensions[1]
        hauteur = dimensions[2]

        perimetre = 2 * (longueur + largeur)
        surface_base = longueur * largeur
        surface_habitable = ratio_efficacite_etage * surface_base
        surface_murs = 2 * hauteur * (longueur + largeur)

        volume_total = surface_base * dimensions[2]
    elif type_batiment == "maison":
        dimensions = demander_dimensions_maison()
        longueur_base = dimensions[0]
        largeur_base = dimensions[1]
        hauteur_base = dimensions[2]
        longueur_toit = dimensions[3]
        largeur_toit = dimensions[4]
        hauteur_toit = dimensions[5]

        surface_base = longueur_base * largeur_base
        perimetre_base = 2 * (longueur_base + largeur_base)
        # approximation générale de la dimension d'un pan du toit
        longueur_pan = math.sqrt((longueur_toit/2)**2 + hauteur_toit**2)
        surface_toit = 2 * longueur_pan * largeur_toit
        perimetre_toit = (2 * longueur_toit) + (2 * largeur_toit)

        # 1 étage = rdc + étage
        nombre_etages = int(input("Nombre étages ? : "))

        perimetre = round(max(perimetre_base, perimetre_toit))
        surface_habitable = round(
        surface_base * ratio_efficacite_etage * nombre_etages)
        volume_total = round((surface_base * hauteur_base) +
                             (surface_toit * hauteur_toit)/2)
        surface_murs = 2 * hauteur_base * (largeur_base + longueur_base)

    else:
        print("Type de bâtiment non pris en charge.")
        return None

    print("Périmètre :", perimetre, "m")
    print("Surface base :", surface_base)
    print("Surface habitable:", surface_habitable, "m²")
    print("Volume :", volume_total, "m³")
    print("Surface murs :", surface_murs, "m²")
    print("\n")

    return type_batiment, perimetre, surface_base, surface_toit, surface_habitable, volume_total, surface_murs, nombre_etages


type_batiment, perimetre, surface_fondations, surface_toit, surface_habitable, volume, surface_murs, nombre_etages = calculer_mesures(
    "maison")

Périmètre : 48 m
Surface base : 112.0
Surface habitable: 186 m²
Volume : 1002 m³
Surface murs : 308.0 m²




1. Ratio d'occupation du sol : Il représente la surface habitable par rapport à la surface totale du terrain sur lequel le bâtiment est construit. Par exemple, si un bâtiment a une surface habitable de 200 m² et est construit sur un terrain de 400 m², le ratio d'occupation du sol serait de 0,5 (200 m² / 400 m²).
2. Ratio d'efficacité de l'étage : Il représente la surface habitable par rapport à la surface totale de chaque étage du bâtiment. Ce ratio est utilisé pour évaluer l'utilisation efficace de l'espace à chaque étage. Par exemple, si un étage a une surface habitable de 500 m² et une surface totale de plancher de 600 m², le ratio d'efficacité de l'étage serait de 0,83 (500 m² / 600 m²).
3. En général, on estime que la surface des fenêtres représente environ 15 % à 20 % de la surface totale des murs.

#### 2) Calcul des prix et quantités de départ puis côuts des matériaux


In [63]:
# Programme de calcul des quantités nécessaires en matériaux pour la construction d'un bâtiment

########### Définition surfaces ############
# taille classique brique 500 mm * 200 mm * 220 mm
surface_cote_brique = 0.5 * 0.220
nombre_brique_m2 = 1 / surface_cote_brique
# taille classique parpaing 500 mm * 200 mm * 250 mm
surface_cote_parpaing = 0.5 * 0.25
nombre_parpaing_m2 = 1 / surface_cote_parpaing 


########### défintion des coûts ############

# prix revêtement
prix_carrelage_m2 = 40
prix_stratifie_m2 = 10
prix_moquette_m2 = 20
prix_parquet_m2 = 70

# prix autres matériaux
prix_m3_beton = 120
prix_brique = 3.90
prix_parpaing = 3.50
prix_peinture_litre = 6

# définition dictionnaire
prix_materiaux = {
    "béton": prix_m3_beton,
    "brique": prix_brique,
    "parpaing": prix_parpaing,
    "peinture": prix_peinture_litre,
    "carrelage": prix_carrelage_m2,
    "stratifie": prix_stratifie_m2,
    "moquette": prix_moquette_m2,
    "parquet": prix_parquet_m2
}


# Fonction pour calculer les quantités nécessaires
def calcul_cout_materiaux(surface_fondations, surface_habitable, surface_murs, type_batiment):
    type_revetement = int(input(
        "Quel revêtement souhaitez-vous équiper votre bâtiment en majorité ? \n - (0) Carrelage \n - (1) Stratifié \n - (2) Moquette \n - (3) Parquet"))
    type_mur = int(input("Souhaitez-vous utiliser : (0) la brique \n - (1) le parpaing : "))

    # Calcul des quantités nécessaires pour chaque matériau
    if type_batiment == "maison":
        quantite_beton = 0.5 * surface_fondations  # m³
        nombre_briques = round(surface_murs * nombre_brique_m2)  # unités
        nombre_parpaings = round(surface_murs * nombre_parpaing_m2) #unités
        quantite_revetement = round(1.10 * surface_habitable)  # m²
        quantite_peinture = surface_habitable * 10  # litres*

    elif type_batiment == "immeuble":
        quantite_beton = 1 * surface_fondations  # m³
        nombre_briques = round(surface_murs * nombre_brique_m2)  # unités
        nombre_parpaings = round(surface_murs * nombre_parpaing_m2) #unités
        quantite_revetement = round(1.10 * surface_habitable)  # m² #prévision de 10% de perte incluse dans le calcul 
        quantite_peinture = surface_habitable * 12  # litres

    cout_briques = round(nombre_briques * prix_materiaux["brique"])
    cout_parpaings = round(nombre_parpaings * prix_materiaux["parpaing"])
    cout_beton = quantite_beton * prix_materiaux["béton"]
    cout_revetement = 0
    if type_revetement == 0:
        cout_revetement += quantite_revetement * prix_materiaux["carrelage"]
    if type_revetement == 1:
        cout_revetement += quantite_revetement * prix_materiaux["stratifie"]
    if type_revetement == 2:
        cout_revetement += quantite_revetement * prix_materiaux["moquette"]
    if type_revetement == 3:
        cout_revetement += quantite_revetement * prix_materiaux["parquet"]

    cout_mur = 0
    if type_mur == 0 :
        cout_mur += cout_briques
    elif type_mur == 1 :
        cout_mur += cout_parpaings
    cout_peinture = quantite_peinture * prix_materiaux["peinture"]
    cout_total = cout_beton + cout_mur + cout_revetement + cout_peinture



    ######### Affichage des résultats ############
    # Quantités
    print("Quantités nécessaires en matériaux :")
    print("-------------------------------")
    print("Quantité béton :", quantite_beton, "m³")
    if type_mur == 0 :
        print("Nombre briques :", nombre_briques, "unités")
    if type_mur == 1 :
        print("Nombre parpaings :", nombre_parpaings, "unités")
    print("Quantité revêtement :", quantite_revetement, "m²")
    print("Quantité peinture:", quantite_peinture, "L")

    # Coûts finaux
    print("\n")
    print("Coûts en matériaux :")
    print("-------------------------------")
    print("Coût béton :", cout_beton, "€")
    if type_mur == 0 :
        print("Coût briques :", cout_briques, "€")
    if type_mur == 1 :
        print("Coût parpaings :", cout_parpaings, "€")

    print("Coût revêtement :", cout_revetement, "€")
    print("Coût peinture:", cout_peinture, "€")
    print("-------------------------------")
    print("Coût total : ", cout_total, "€")



    return cout_beton, cout_briques, cout_revetement, cout_peinture


# On garde le résultat 
cout_materiaux = calcul_cout_materiaux(surface_fondations, surface_habitable, surface_murs, "maison")


Quantités nécessaires en matériaux :
-------------------------------
Quantité béton : 56.0 m³
Nombre briques : 2800 unités
Quantité revêtement : 205 m²
Quantité peinture: 1860 L


Coûts en matériaux :
-------------------------------
Coût béton : 6720.0 €
Coût briques : 10920 €
Coût revêtement : 4100 €
Coût peinture: 11160 €
-------------------------------
Coût total :  32900.0 €


1. Béton : Le béton est souvent utilisé pour les fondations, les dalles de plancher, les piliers, etc. L'estimation de la quantité de béton nécessaire dépend de l'épaisseur des éléments en béton. On peut estimer qu'environ 0,5 à 1 mètre cube de béton est nécessaire par mètre carré de dalle ou de fondation.
2. Briques ou blocs : Si vous utilisez des briques ou des blocs pour les murs, vous pouvez estimer qu'environ 60 à 70 briques ou blocs pleins sont nécessaires par mètre carré de mur. Cette estimation peut varier en fonction de la taille et du type de briques ou de blocs utilisés.
3. Charpente en bois : L'estimation de la quantité de bois nécessaire dépend de la conception de la charpente et de la taille du bâtiment. Il est préférable de consulter un ingénieur ou un architecte pour obtenir une estimation précise en fonction des plans de construction.
4. Revêtements de sol : La quantité de revêtement de sol nécessaire dépend du type de revêtement choisi (carrelage, parquet, etc.) et de la superficie des pièces. En général, il est recommandé d'ajouter une marge supplémentaire d'environ 10 % pour les pertes et les coupes.
5. Peinture : La quantité de peinture nécessaire dépend de la surface à peindre, du nombre de couches et de la qualité de la peinture. Un litre de peinture peut généralement couvrir environ 10 à 12 mètres carrés de surface, en fonction de la porosité des murs.

#### 3) Comprendre les coûts de la main d’oeuvre


La compréhension des coûts matériels et de main-d'œuvre est essentielle pour établir un devis précis en construction. Les ajustements sur les coûts des matériaux, de la main-d'œuvre et de l'équipement sont nécessaires. Cela inclut l'ajout de taxes de vente et un pourcentage supplémentaire pour couvrir le gaspillage des matériaux.

Estimer avec précision le gaspillage des matériaux nécessite de l'expérience. Ajouter un pourcentage supplémentaire au coût total des matériaux permet de tenir compte du gaspillage prévu. Cela garantit une marge adéquate et la disponibilité des matériaux nécessaires.

Outre les coûts matériels, il est crucial de considérer les coûts de main-d'œuvre. Cela implique les salaires, les taxes, les assurances et les avantages sociaux. Il faut calculer ces coûts précisément pour établir un budget réaliste. La productivité de la main-d'œuvre et la taille de l'équipe doivent également être prises en compte pour maximiser l'efficacité des travaux.

#### 4) Requis généraux


#### 5) Conditions existantes du lieu
Simplification : le terrain détruit est constructible  

#### 6) Maçonnerie


* Réalisation des fondations : 60 euros par mètre carré.
* Réalisation du soubassement : 60 euros par mètre carré.
* Maçonnerie du rez-de-chaussée : 90 euros par mètre carré.
* Maçonnerie de l'étage : 120 euros par mètre carré.
* Conception d'une charpente et autres travaux : 150 euros par mètre carré.

In [65]:
def calculer_cout_maconnerie(surface_fondations, nombre_etages, type_batiment):
    cout_fondations = 60 * surface_fondations
    cout_soubassement = 60 * surface_fondations
    cout_rez_de_chaussee = 90 * surface_fondations

    #si nombre_etages > 2, multiplier cout_etage par nombre_etages
    cout_etage = 0
    if nombre_etages > 2 :
        cout_etage += nombre_etages * 120 * surface_fondations

    cout_etage = 120 * surface_fondations

    cout_charpente = 0 # immeuble -> cout_charpente = 0
    if type_batiment == "maison":
        cout_charpente += 150 * surface_fondations

    print("-------------------------------")
    print("Prix fondations : ", cout_fondations, "€")
    print("Prix soubassement : ", cout_soubassement, "€")
    print("Prix rez-de-chaussée : ", cout_rez_de_chaussee, "€")
    print("Prix étages : ", cout_etage, "€")
    print("Prix charpente : ", cout_charpente, "€")
    print("-------------------------------")

    cout_total_maconnerie = round(cout_rez_de_chaussee + cout_etage + cout_fondations + cout_soubassement + cout_charpente)
    print("Le coût de la maçonnerie est de", cout_maconnerie, "euros.")

    return cout_total_maconnerie
    
# On garde le résultat 
cout_maconnerie = calculer_cout_maconnerie(surface_fondations, nombre_etages, type_batiment)



-------------------------------
Prix fondations :  6720.0 €
Prix soubassement :  6720.0 €
Prix rez-de-chaussée :  10080.0 €
Prix étages :  13440.0 €
Prix charpente :  16800.0 €
-------------------------------
Le coût de la maçonnerie est de 53760 euros.


#### 7) Métaux pour la structure

* Divers 
* Poutres 
* Tôit en métal
* Métaux pour la soudure
* Items fournis par le maçon 

In [ ]:
def calcul_prix_metaux(type_batiment, surface_fondations, surface_toit):
    

#### 8) Plastiques, bois, composites


#### 9) Protections thermiques et contre la moisissure


#### 10) Ouvertures (portes, fenêtres)


#### 11) Finitions


#### 12) Système anti-incendies


#### 13) Plomberie


#### 14) Chauffage, ventilation, climatisation


#### 15) Système électrique


#### 16) Travail du terrain, préparation du terrain, irrigation


#### 17) Amélioration extérieures


#### 18) Equipements


#### 19) Coût final des Consultations Professionnels 


* Consulation architecte : de 5 à 16% du total
* Au delà d'un total de 1 000 000 € , 4% pour le management de l'équipe 
* Consulations des ingénieurs : 0.5% du total
* Modèlisation des plans 1000 € environ 

### TOTAL

# Coûts de destruction


In [ ]:
# les indices sont entre 1-3
def calculer_cout_demolition(surface, nombre_etages, indice_complexite, indice_etat):
    # Coûts initiaux par unité de surface
    cout_par_metre_carre = 100  # Coût de base par mètre carré

    # Facteurs multiplicateurs en fonction de la complexité et de l'état du bâtiment
    facteur_complexite = [1, 1.2, 1.5]  # Indice de complexité (1-3)
    facteur_etat = [1, 1.1, 1.3]  # Indice d'état (1-3)

    # Calcul des coûts de base
    cout_base = surface * cout_par_metre_carre

    # Calcul des coûts en fonction de la complexité et de l'état du bâtiment
    cout_total = cout_base * \
        facteur_complexite[indice_complexite-1] * facteur_etat[indice_etat-1]

    # Coûts supplémentaires en fonction du nombre d'étages
    cout_supplementaire_etages = 5000 * nombre_etages

    # Ajout des coûts supplémentaires
    cout_total += cout_supplementaire_etages

    return round(cout_total)

In [ ]:
# Demande des informations à l'utilisateur

surface = 200  # surface du bâtiment cumulée  (en mètres carrés)
nombre_etages = 2      # nombre d'étages du bâtiment)
# indice de complexité du bâtiment (de indice_standing à 3)
indice_complexite = 2
indice_etat = 1  # indice de l'état du bâtiment (de indice_standing à 3)

# Appel de la fonction pour calculer les coûts de démolition
cout_demolition = calculer_cout_demolition(
    surface, nombre_etages, indice_complexite, indice_etat)

# Affichage du résultat
print("Le coût de démolition du bâtiment est de", cout_demolition, "euros.")

# Coûts de réhabilitation


### Définition des données dans un dictionnaire


In [ ]:
# Problèmes de construction
import math
geotechnical_investigation_prices = {
    "étude topographique": 2.16,  # € par mètre carré
    "rapport ingénieur": 2100  # €
}

demolition_prices = {
    "lavabo": 51,  # € chacun
    "baignoire": 82,  # € chacune
    "douche": 137,  # € chacune
    "lampe incandescente": 17,  # € chacune
    "lave-vaisselle": 31,  # € chacun
    "porte": 55,  # € chacune
    "fenêtre": 56,  # € chacune
    "poutre": 16.40,  # € par mètre linéaire
    "cloison en plâtre": 4.95,  # € par mètre carré
    "charpente": 11.08,  # € par mètre linéaire
    "tuiles": 15.50,  # € par mètre carré
    "murs": 43.92,  # € par mètre carré
    "murs porteurs": 174.04,  # € par mètre carré
    "sol en béton": 48.70,  # € par mètre carré
    "location camion hydraulique 12T": 1400  # € par jour
}

# Dégats des eaux
mold_prices = {
    "inspection de la zone": 481,  # €
    "réduction de la moisissure": 1923,  # €
    "enlèvement de zones contaminées par la moisissure": 18132  # €
}

mold_demolition_prices = {
    "sol": 3.29,  # € par pied carré (10.79 € par mètre carré)
    "plafond": 2.17  # € par pied carré (7.12 € par mètre carré)
}

# Problèmes électriques
electrical_replacement_prices = {
    "remplacement du panneau électrique": (879, 2747.25),  # €
    "réparation ou remplacement des fils et des câbles": (546, 1638),  # €
    "installation de nouveaux circuits électriques": (546, 1638),  # €
    "installation de systèmes d'éclairage": (227, 1137)  # € par pièce
}

electrical_upgrade_prices = {
    # € par prise ou interrupteur
    "mise à niveau des prises de courant et des interrupteurs": (72.50, 108.75),
    "mise à jour des systèmes de sécurité": (546, 2185)  # €
}

# Problèmes de plomberie
bathroom_prices = {
    "lavabo": 60.06,  # €
    "baignoire/douche": 150.15,  # €
    "toilette": (200, 600),  # €
    "robinet de douche": (60, 120),  # €
    "robinet de lavabo": (30, 100)  # €
}

plumbing_repair_prices = {
    "réparation d'une fuite": (200, 550),  # €
    "remplacement d'un tuyau": (100, 200),  # €
    "remplacement d'un robinet": (100, 250),  # €
    "débouchage d'un évier": (100, 200),  # €
    "remplacement d'un évier": (200, 500)  # €
}

# Problème de chauffage, ventilation, climatisation
hvac_maintenance_prices = {
    "entretien annuel d'une chaudière": (120, 200),  # €
    "entretien du système électronique": 74.62,  # €
    "entretien d'une pompe à chaleur": (100, 300),  # €
    "nettoyage des conduits de ventilation": (200, 500)  # €
}

hvac_replacement_repair_prices = {
    "ventilation + conduits": 546,  # €
    "chaudière": [1500, 3000, 4000, 7000],  # €
    # €
    "installation d'un nouveau système de chauffage (pompe à chaleur)": (5000, 12000),
    "réparation ou remplacement de la climatisation": (1000, 5000),  # €
    # €
    "installation d'un système de ventilation mécanique contrôlée (VMC)": (1000, 4000),
    # €
    "installation d'un système de climatisation centralisée": (3000, 8000)
}

# Problème d'isolation
insulation_prices = {
    "isolation des combles": (20, 80),  # € par mètre carré
    "isolation des murs": (50, 150),  # € par mètre carré
    "isolation des planchers": (30, 80),  # € par mètre carré
    "installation de fenêtres isolantes": (300, 1000),  # € par fenêtre
    "isolation des portes": (50, 500)  # € par porte
}

# Présence de nuisibles
pest_control_prices = {
    "élimination de souris et de rats": (150, 500),  # €
    "élimination de punaises de lit": (200, 1000),  # €
    "élimination des guêpes et des nids de frelons": (60, 250),  # €
    "élimination des fourmis": (70, 200)  # €
}

pest_damage_repair_prices = {
    "réparation des dommages causés par les rongeurs": (150, 500),  # €
    "réparation des dommages causés par les insectes": (200, 1000),  # €
    # € par mètre carré
    "nettoyage et désinfection de la zone touchée": (50, 100)
}

# Problèmes d'humidité
humidity_detection_treatment_prices = {
    "diagnostic de l'humidité": (80, 150),  # €
    # € par mètre linéaire
    "traitement de l'humidité par injection": (100, 150),
    "traitement de l'humidité par ventilation": (1500, 5000),  # €
    "traitement de l'humidité par drainage": (1000, 5000)  # €
}

# Problèmes de toiture
roof_repair_prices = {
    "remplacement de tuiles": (2, 4),  # € par tuile
    "réparation de fuite": (200, 400),  # €
    "remplacement du revêtement de toit": (40, 100),  # € par mètre carré
    "nettoyage de la toiture": (200, 500)  # €
}

roof_replacement_prices = {
    "toiture en tuiles": (60, 150),  # € par mètre carré
    "toiture en ardoise": (80, 250),  # € par mètre carré
    "toiture en zinc": (120, 200),  # € par mètre carré
    "toiture en bac acier": (30, 100)  # € par mètre carré
}

# Problèmes de fondation
foundation_crack_repair_prices = {
    # € par fissure
    "réparation de fissures - injection de résine époxy": (100, 300),
    "ancrage de la fondation": (2000, 5000)  # €
}

foundation_subsidence_repair_prices = {
    "injection de mousse polyuréthane": (1000, 3000),  # €
    "remplacement des pieux": (3000, 7000)  # €
}

foundation_wall_repair_prices = {
    # € par mètre carré
    "rénovation de mur en blocs de béton": (100, 150),
    # € par mètre carré
    "réparation de murs de fondation en briques": (100, 250),
    "rénovation de mur en béton coulé": (300, 500)  # € par mètre carré
}

# Problèmes de construction
geotechnical_investigation_prices = {
    "étude topographique": 2.16,  # € par mètre carré
    "rapport ingénieur": 2100  # €
}

demolition_prices = {
    "lavabo": 51,  # € chacun
    "baignoire": 82,  # € chacune
    "douche": 137,  # € chacune
    "lampe incandescente": 17,  # € chacune
    "lave-vaisselle": 31,  # € chacun
    "porte": 55,  # € chacune
    "fenêtre": 56,  # € chacune
    "poutre": 16.40,  # € par mètre linéaire
    "cloison en plâtre": 4.95,  # € par mètre carré
    "charpente": 11.08,  # € par mètre linéaire
    "tuiles": 15.50,  # € par mètre carré
    "murs": 43.92,  # € par mètre carré
    "murs porteurs": 174.04,  # € par mètre carré
    "sol en béton": 48.70,  # € par mètre carré
    "location camion hydraulique 12T": 1400  # € par jour
}

# Dégats des eaux
mold_prices = {
    "inspection de la zone": 481,  # €
    "réduction de la moisissure": 1923,  # €
    "enlèvement de zones contaminées par la moisissure": 18132  # €
}

mold_demolition_prices = {
    "sol": 3.29,  # € par pied carré (10.79 € par mètre carré)
    "plafond": 2.17  # € par pied carré (7.12 € par mètre carré)
}

# Problèmes électriques
electrical_replacement_prices = {
    "remplacement du panneau électrique": (879, 2747.25),  # €
    "réparation ou remplacement des fils et des câbles": (546, 1638),  # €
    "installation de nouveaux circuits électriques": (546, 1638),  # €
    "installation de systèmes d'éclairage": (227, 1137)  # € par pièce
}

electrical_upgrade_prices = {
    # € par prise ou interrupteur
    "mise à niveau des prises de courant et des interrupteurs": (72.50, 108.75),
    "mise à jour des systèmes de sécurité": (546, 2185)  # €
}

# Problèmes de plomberie
bathroom_prices = {
    "lavabo": 60.06,  # €
    "baignoire/douche": 150.15,  # €
    "toilette": (200, 600),  # €
    "robinet de douche": (60, 120),  # €
    "robinet de lavabo": (30, 100)  # €
}

plumbing_repair_prices = {
    "réparation d'une fuite": (200, 550),  # €
    "remplacement d'un tuyau": (100, 200),  # €
    "remplacement d'un robinet": (100, 250),  # €
    "débouchage d'un évier": (100, 200),  # €
    "remplacement d'un évier": (200, 500)  # €
}

# Problème de chauffage, ventilation, climatisation
hvac_maintenance_prices = {
    "entretien annuel d'une chaudière": (120, 200),  # €
    "entretien du système électronique": 74.62,  # €
    "entretien d'une pompe à chaleur": (100, 300),  # €
    "nettoyage des conduits de ventilation": (200, 500)  # €
}

hvac_replacement_repair_prices = {
    "ventilation + conduits": 546,  # €
    "chaudière": [1500, 3000, 4000, 7000],  # €
    # €
    "installation d'un nouveau système de chauffage (pompe à chaleur)": (5000, 12000),
    "réparation ou remplacement de la climatisation": (1000, 5000),  # €
    # €
    "installation d'un système de ventilation mécanique contrôlée (VMC)": (1000, 4000),
    # €
    "installation d'un système de climatisation centralisée": (3000, 8000)
}

# Problème d'isolation
insulation_prices = {
    "isolation des combles": (20, 80),  # € par mètre carré
    "isolation des murs": (50, 150),  # € par mètre carré
    "isolation des planchers": (30, 80),  # € par mètre carré
    "installation de fenêtres isolantes": (300, 1000),  # € par fenêtre
    "isolation des portes": (50, 500)  # € par porte
}

# Présence de nuisibles
pest_control_prices = {
    "élimination de souris et de rats": (150, 500),  # €
    "élimination de punaises de lit": (200, 1000),  # €
    "élimination des guêpes et des nids de frelons": (60, 250),  # €
    "élimination des fourmis": (70, 200)  # €
}

pest_damage_repair_prices = {
    "réparation des dommages causés par les rongeurs": (150, 500),  # €
    "réparation des dommages causés par les insectes": (200, 1000),  # €
    # € par mètre carré
    "nettoyage et désinfection de la zone touchée": (50, 100)
}

# Problèmes d'humidité
humidity_detection_treatment_prices = {
    "diagnostic de l'humidité": (80, 150),  # €
    # € par mètre linéaire
    "traitement de l'humidité par injection": (100, 150),
    "traitement de l'humidité par ventilation": (1500, 5000),  # €
    "traitement de l'humidité par drainage": (1000, 5000)  # €
}

# Problèmes de toiture
roof_repair_prices = {
    "remplacement de tuiles": (2, 4),  # € par tuile
    "réparation de fuite": (200, 400),  # €
    "remplacement du revêtement de toit": (40, 100),  # € par mètre carré
    "nettoyage de la toiture": (200, 500)  # €
}

roof_replacement_prices = {
    "toiture en tuiles": (60, 150),  # € par mètre carré
    "toiture en ardoise": (80, 250),  # € par mètre carré
    "toiture en zinc": (120, 200),  # € par mètre carré
    "toiture en bac acier": (30, 100)  # € par mètre carré
}

# Problèmes de fondation
foundation_crack_repair_prices = {
    # € par fissure
    "réparation de fissures - injection de résine époxy": (100, 300),
    "ancrage de la fondation": (2000, 5000)  # €
}

foundation_subsidence_repair_prices = {
    "injection de mousse polyuréthane": (1000, 3000),  # €
    "remplacement des pieux": (3000, 7000)  # €
}

foundation_wall_repair_prices = {
    # € par mètre carré
    "rénovation de mur en blocs de béton": (100, 150),
    # € par mètre carré
    "réparation de murs de fondation en briques": (100, 250),
    "rénovation de mur en béton coulé": (300, 500)  # € par mètre carré
}

### Calcul du coût de réhabilitation


In [ ]:
def calcul_cout_rehabilitation(surface_terrain, salon_surface, chambres_surface,  salle_de_bain_surface, cuisine_surface, salle_a_manger_surface, nombre_chambres, indice_standing):

    ############ Define variables ##############
    surface_tuile = 0.16  # constant
    probleme_terrain = 0
    renovation_interieure = 0
    renovation_salon = 0
    renovation_chambres = 0
    renovation_salle_de_bain = 0
    renovation_cuisine = 0
    renovation_salle_a_manger = 0
    renovation_toiture = 0
    surface_toiture = 0
    surface_toiture_reparation = 0
    remplacement_toiture = 0
    remplacement_toiture_materiau = 0
    reparation_charpente = 0
    probleme_moisissure = 0
    probleme_moisissure_plafond = 0
    probleme_moisissure_sol = 0
    probleme_moisissure_gravite = 0
    probleme_electriques = 0
    probleme_norme_electriques = 0
    probleme_plomberie = 0
    probleme_plomberie_indice = 0
    probleme_hvac = 0
    probleme_hvac_indice = 0
    choix_chauffage = 0
    reparation_clim = 0
    installation_vmc = 0
    probleme_isolation = 0
    probleme_isolation_murs = 0
    probleme_isolation_murs_surface = 0
    probleme_isolation_combles = 0
    probleme_isolation_combles_surface = 0
    probleme_isolation_plancher = 0
    probleme_isolation_plancher_surface = 0
    probleme_nuisibles = 0
    probleme_rats = 0
    probleme_punaises = 0
    probleme_guepes = 0
    probleme_fourmis = 0
    probleme_nuisibles_reparation = 0
    probleme_nuisibles_surface = 0
    probleme_humidite = 0
    probleme_humidite_traitement = 0
    probleme_fondations = 0
    probleme_fondations_fissure = 0
    probleme_fondations_affaissement = 0
    remplacement_fondations_pieux = 0
    probleme_fondations_murs = 0
    probleme_fondations_murs_type = 0
    probleme_fondations_murs_surface = 0

    ############ User input data ############

    ### Input terrain ###
    print("""répondez : (oui -> 1), (non -> 0)""")
    probleme_terrain = int(
        input("Votre logement a-t'il des problèmes de terrain ? : "))
    renovation_interieure = int(
        input("Voulez-vous rénover votre intérieur ? : "))
    if renovation_interieure:
        renovation_salon = int(input("Voulez-vous rénover votre salon ? : "))
        renovation_chambres = int(
            input("Voulez-vous rénover vos chambres ? : "))
        renovation_salle_de_bain = int(
            input("Voulez-vous rénover votre salle de bain ? : "))
        renovation_cuisine = int(
            input("Voulez-vous rénover votre cuisine ? : "))
        renovation_salle_a_manger = int(
            input("Voulez-vous rénover votre salle à manger ? : "))

    ### Input toiture ###
    renovation_toiture = int(input("Avez vous des problèmes de toiture ? : "))
    if renovation_toiture:
        surface_toiture = int(
            input("quelle est la surface de votre toiture ? : "))
        surface_toiture_reparation = int(
            input("Quelle est la surface à faire réparer ? : "))
        remplacement_toiture = int(
            input("Souhaitez-vous plutôt changer l'intégralité de votre toiture ? : "))
        remplacement_toiture_materiau = int(input(
            "Quel matériau souhaitez-vous utiliser ? : \n - (0) tuiles \n - (1) ardoise \n - (2) zinc \n - (3) bac acier : "))
        reparation_charpente = int(
            input("devez-vous changer votre charpente ? : "))

    ### Input moisissurre ###
    probleme_moisissure = int(
        input("avez-vous des problèmes de moisissure ? : "))
    if probleme_moisissure:
        probleme_moisissure_plafond = int(
            input("Est-ce que cela concerne le plafond ? : "))
        probleme_moisissure_sol = int(
            input("Est-ce que cela concerne le sol ? : "))
        probleme_moisissure_gravite = int(
            input("Est-ce que la moisissure occupe une pièce entière ? "))

    ### Input elec ###
    probleme_electriques = int(
        input("Est-ce que vous avez des problèmes électriques ? : "))
    if probleme_electriques:
        probleme_norme_electriques = int(
            input("Vos installations électriques sont-elles aux normes ? : "))
    probleme_plomberie = int(
        input("Est-ce que vous avez des problèmes de promberie ? : "))
    if probleme_plomberie:
        probleme_plomberie_indice = int(
            input("Sont-ils graves : Non -> 0 , Oui -> 1 ? : "))
    probleme_hvac = int(
        input("Avez-vous des problèmes de ventilation, chauffage ? : "))

    ### Input HVAC ###
    if probleme_hvac:
        probleme_hvac_indice = int(
            input("Quelle gamme de chaudière souhaitez-vous ? : de 0 à 3 : "))
        choix_chauffage = int(
            input("Préférez-vous une chaudière (0) ou une pompe à chaleur (1) ? : "))
        reparation_clim = int(
            input("Devez-vous réparer/installer votre climatisation ? : "))
        installation_vmc = int(input("Souhaitez-vous installer une VMC ? : "))

    ### Input isolation ###
    probleme_isolation = int(input("Avez-vous des problemes d'isolation ? : "))
    if probleme_isolation:
        probleme_isolation_murs = int(
            input("Souhaitez-vous isoler vos murs ? : "))
        probleme_isolation_murs_surface = int(
            input("Quelle surface devez-vous isoler ? : "))
        probleme_isolation_combles = int(
            input("Souhaitez-vous isoler vos combles ? : "))
        probleme_isolation_combles_surface = int(
            input("Quelle surface devez-vous isoler ? : "))
        probleme_isolation_plancher = int(
            input("Souhaitez-vous isoler votre plancher ? : "))
        probleme_isolation_plancher_surface = int(
            input("Quelle surface devez-vous isoler ? : "))

    ### Input nuisibles ###
    probleme_nuisibles = int(
        input("Avez-vous des problèmes de nuisibles ? : "))
    if probleme_nuisibles:
        probleme_rats = int(input("Etes-vous infesté par les rats ? : "))
        probleme_punaises = int(
            input("Etes-vous infesté par les punaises de lit : "))
        probleme_guepes = int(input("Etes-vous infesté par les guèpes ? : "))
        probleme_fourmis = int(
            input("Etes-vous infesté par les fourmis ? : "))
        probleme_nuisibles_reparation = int(
            input("Avez-vous des dommages causés par ces nuisibles ? : "))
        probleme_nuisibles_surface = int(
            input("Quelle est la surface concernée ? : "))
        probleme_humidite = int(
            input("Avez-vous des problèmes d'humidité ? : "))
        probleme_humidite_traitement = int(input(
            "Vous souhaitez utiliser la méthode : \n -(0) traitement par injection \n - (1) traitement par ventilation \n - (2) traitement par drainage : "))

    ### Input fondations ###
    probleme_fondations = int(
        input("Vos fondations sont-elles endommagées ? : "))
    if probleme_fondations:
        probleme_fondations_fissure = int(
            input("Vos fondations sont-elles fissurées ? : "))
        probleme_fondations_affaissement = int(
            input("Vos fondations se sont-elles affaissées ? : "))
        remplacement_fondations_pieux = int(
            input("Devez-vous changer les pieux de vos fondations ? : "))
        probleme_fondations_murs = int(
            input("Vos murs de fondations sont-ils endommagés ? : "))
        if probleme_fondations_murs:
            probleme_fondations_murs_surface = int(
                input("Quelle est la surface du mur à réparer ? : "))
            probleme_fondations_murs_type = int(input(
                "Choisissez le type de mur à rénover: \n - (0) Mur en bloc de béton \n - (1) Mur en briques \n - (2) Mur en béton coulé : "))

    ############ Coût des problèmes de terrain ############
    geotechnical_investigation_cost = 0
    if probleme_terrain:
        geotechnical_investigation_cost += surface_terrain * \
            geotechnical_investigation_prices["étude topographique"]
        geotechnical_investigation_cost += geotechnical_investigation_prices["rapport ingénieur"]

    print("Terrain : ", geotechnical_investigation_cost)

    ############ Coût démoliton pièces, murs, etc ############
    destruction_cost = 0

    if renovation_salon:
        destruction_cost += demolition_prices["porte"] + demolition_prices["fenêtre"] + demolition_prices["cloison en plâtre"] + \
            demolition_prices["murs porteurs"]*math.sqrt(
                salon_surface) * 2  # utilisation racine pour calculer longueur du mur + hauteur par défaut de 2m
    if renovation_chambres:
        destruction_cost += (demolition_prices["lampe incandescente"] +
                             demolition_prices["porte"] + demolition_prices["fenêtre"]) * nombre_chambres

    if renovation_salle_de_bain:
        destruction_cost += demolition_prices["lavabo"] + demolition_prices["baignoire"] + \
            demolition_prices["douche"] + \
            demolition_prices["porte"] + demolition_prices["fenêtre"]

    if renovation_cuisine:
        destruction_cost += demolition_prices["lavabo"] + \
            demolition_prices["lampe incandescente"] + \
            demolition_prices["lave-vaisselle"]

    if renovation_salle_a_manger:
        demolition_prices["lampe incandescente"] + demolition_prices["porte"] + demolition_prices["cloison en plâtre"] * \
            math.sqrt(salle_a_manger_surface) * \
            2  # utilisation racine pour calculer longueur du mur + hauteur par défaut de 2m

    if renovation_toiture:
        if reparation_charpente:
            destruction_cost += demolition_prices["charpente"]*surface_toiture
        destruction_cost += demolition_prices["tuiles"]*surface_toiture

    print("Demolition pièces : ", destruction_cost)

    ############ Coût des problèmes liés à la moisissure ############
    mold_cost = 0
    if probleme_moisissure:
        mold_cost += mold_prices["inspection de la zone"] + \
            mold_prices["réduction de la moisissure"]
    if probleme_moisissure_gravite:
        mold_cost += mold_prices["enlèvement de zones contaminées par la moisissure"]
    if probleme_moisissure_sol:
        mold_cost += mold_demolition_prices["sol"] * salle_de_bain_surface
    if probleme_moisissure_plafond:
        mold_cost += mold_demolition_prices["plafond"] * salle_de_bain_surface

    print("Moisissure : ", mold_cost)

    ############ Coût des problèmes électriques ############

    electrical_cost = 0
    if probleme_electriques:
        electrical_cost += electrical_replacement_prices["remplacement du panneau électrique"][indice_standing] + electrical_replacement_prices["réparation ou remplacement des fils et des câbles"][indice_standing] + \
            electrical_replacement_prices["installation de nouveaux circuits électriques"][indice_standing] + \
            electrical_replacement_prices["installation de systèmes d'éclairage"][indice_standing]

        if not probleme_norme_electriques:
            electrical_cost += electrical_upgrade_prices["mise à niveau des prises de courant et des interrupteurs"][indice_standing] + \
                electrical_upgrade_prices["mise à jour des systèmes de sécurité"][indice_standing]

    print("Elec : ", electrical_cost)

    ############ Réparation plomberie ############

    plumbing_cost = 0
    if renovation_salle_de_bain:
        plumbing_cost += bathroom_prices["lavabo"] + bathroom_prices["baignoire/douche"] + bathroom_prices["toilette"][indice_standing] + \
            bathroom_prices["robinet de douche"][indice_standing] + \
            bathroom_prices["robinet de lavabo"][indice_standing]

    if probleme_plomberie:
        plumbing_cost += plumbing_repair_prices["remplacement d'un tuyau"][indice_standing] + plumbing_repair_prices["remplacement d'un robinet"][indice_standing] + \
            plumbing_repair_prices["débouchage d'un évier"][indice_standing] + plumbing_repair_prices["remplacement d'un évier"][indice_standing] + \
            plumbing_repair_prices["réparation d'une fuite"][probleme_plomberie_indice]

    print("Plomberie : ", plumbing_cost)

    ############## HVAC ##############
    hvac_cost = 0
    if probleme_hvac:
        hvac_cost += hvac_replacement_repair_prices["ventilation + conduits"]
        if choix_chauffage:
            hvac_cost += hvac_replacement_repair_prices["chaudière"][probleme_hvac_indice]
        else:
            hvac_cost += hvac_replacement_repair_prices[
                "installation d'un nouveau système de chauffage (pompe à chaleur)"][indice_standing]
        if reparation_clim:
            hvac_cost += hvac_replacement_repair_prices["réparation ou remplacement de la climatisation"][indice_standing]
        if installation_vmc:
            hvac_cost += hvac_replacement_repair_prices[
                "installation d'un système de ventilation mécanique contrôlée (VMC)"][indice_standing]

    print("HVAC : ", hvac_cost)

    ######## Coût isolation ########
    insulation_cost = 0

    if probleme_isolation_combles:
        insulation_cost += insulation_prices["isolation des combles"][indice_standing] * \
            probleme_isolation_combles_surface
    if probleme_isolation_murs:
        insulation_cost += insulation_prices["isolation des murs"][indice_standing] * probleme_isolation_murs_surface + 2 * \
            insulation_prices["isolation des portes"][indice_standing] + 2 * \
            insulation_prices["installation de fenêtres isolantes"][indice_standing]
    if probleme_isolation_plancher:
        insulation_cost += insulation_prices["isolation des planchers"][indice_standing] * \
            probleme_isolation_plancher_surface

    print("Isolation : ", insulation_cost)

    ######### Pest #########
    pest_cost = 0
    if probleme_guepes:
        pest_cost += pest_control_prices["élimination des guêpes et des nids de frelons"][indice_standing]
        if probleme_nuisibles_reparation:
            pest_cost += pest_damage_repair_prices["réparation des dommages causés par les insectes"][indice_standing] + \
                probleme_nuisibles_surface * \
                pest_damage_repair_prices["nettoyage et désinfection de la zone touchée"][indice_standing]
    if probleme_punaises:
        pest_cost += pest_control_prices["élimination de punaises de lit"][indice_standing]
    if probleme_fourmis:
        pest_cost += pest_control_prices["élimination des fourmis"][indice_standing]
    if probleme_rats:
        pest_cost += pest_control_prices["élimination de souris et de rats"][indice_standing]
        if probleme_nuisibles_reparation:
            pest_cost += pest_damage_repair_prices["réparation des dommages causés par les rongeurs"][indice_standing] + \
                probleme_nuisibles_surface * \
                pest_damage_repair_prices["nettoyage et désinfection de la zone touchée"][indice_standing]

    print("Nuisibles : ", pest_cost)

    ########## Problèmes d'humidité #########
    humidity_treatment_cost = 0
    if probleme_humidite:
        humidity_treatment_cost += humidity_detection_treatment_prices["diagnostic de l'humidité"] * round(
            math.sqrt(salle_de_bain_surface))
        if probleme_humidite_traitement == 0:
            humidity_treatment_cost += humidity_detection_treatment_prices[
                "traitement de l'humidité par injection"][indice_standing]
        if probleme_humidite_traitement == 1:
            humidity_treatment_cost += humidity_detection_treatment_prices[
                "traitement de l'humidité par ventilation"][indice_standing]
        if probleme_humidite_traitement == 2:
            humidity_treatment_cost += humidity_detection_treatment_prices[
                "traitement de l'humidité par drainage"][indice_standing]

    print("Humidité : ", humidity_treatment_cost)

    ########## Problèmes toiture ########
    roof_repair_cost = 0
    if not remplacement_toiture and renovation_toiture:
        roof_repair_cost += int(surface_tuile * float(surface_toiture_reparation)) * roof_repair_prices["remplacement de tuiles"][indice_standing] + roof_repair_prices["réparation de fuite"][
            indice_standing] + surface_toiture_reparation * roof_repair_prices["remplacement du revêtement de toit"][indice_standing] + roof_repair_prices["nettoyage de la toiture"][indice_standing]
    if remplacement_toiture:
        roof_repair_cost += roof_repair_prices["remplacement du revêtement de toit"][indice_standing]*surface_toiture
        if remplacement_toiture_materiau == 0:
            roof_repair_cost += surface_toiture * \
                roof_replacement_prices["toiture en tuiles"][indice_standing]
        if remplacement_toiture_materiau == 1:
            roof_repair_cost += surface_toiture * \
                roof_replacement_prices["toiture en ardoise"][indice_standing]
        if remplacement_toiture_materiau == 2:
            roof_repair_cost += surface_toiture * \
                roof_replacement_prices["toiture en zinc"][indice_standing]
        if remplacement_toiture_materiau == 3:
            roof_repair_cost += surface_toiture * \
                roof_replacement_prices["toiture en bac acier"][indice_standing]

    print("Toiture: ", roof_repair_cost)

    ###### Coût problèmes fondations ######
    foundation_repair_cost = 0
    if probleme_fondations:
        if probleme_fondations_fissure:
            foundation_repair_cost += foundation_crack_repair_prices["réparation de fissures - injection de résine époxy"][indice_standing] + \
                foundation_crack_repair_prices["ancrage de la fondation"][indice_standing]
        if probleme_fondations_affaissement:
            foundation_repair_cost += foundation_subsidence_repair_prices[
                "injection de mousse polyuréthane"][indice_standing]
            if remplacement_fondations_pieux:
                foundation_repair_cost += foundation_subsidence_repair_prices["remplacement des pieux"][indice_standing]

        if probleme_fondations_murs:
            if probleme_fondations_murs_type == 0:
                foundation_repair_cost += foundation_wall_repair_prices[
                    "rénovation de mur en blocs de béton"][indice_standing] * probleme_fondations_murs_surface
            if probleme_fondations_murs_type == 1:
                foundation_repair_cost += foundation_wall_repair_prices[
                    "réparation de murs de fondation en briques"][indice_standing] * probleme_fondations_murs_surface
            if probleme_fondations_murs_type == 2:
                foundation_repair_cost += foundation_wall_repair_prices[
                    "rénovation de mur en béton coulé"][indice_standing] * probleme_fondations_murs_surface

    print("Fondations : ", foundation_repair_cost)

    ###### Somme coût total ######
    total_cost = geotechnical_investigation_cost + destruction_cost + mold_cost + electrical_cost + \
        plumbing_cost + hvac_cost + insulation_cost + \
        pest_cost + roof_repair_cost + foundation_repair_cost

    print("Total : ")
    return total_cost

### Test de la fonction


In [ ]:
print(calcul_cout_rehabilitation(2000, 10, 10, 10, 10, 10, 0, 1))